In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import col, lit

# Paths
source_path = "/Volumes/sivaadbuc/default/batch18test/customers_raw"
target_path = "/Volumes/sivaadbuc/default/batch18test/customers_scd2"

# Step 1: Load incoming batch (CSV)
incoming_df = (spark.read.format("csv")
                        .option("header", "true")
                        .load(source_path)
                        .withColumn("customer_id", col("customer_id").cast("int"))
                        .withColumn("update_timestamp", col("update_timestamp").cast("timestamp"))
                        .select("customer_id", "name", "city", "update_timestamp"))

# Step 2: Create Delta target table if not exists
try:
    target = DeltaTable.forPath(spark, target_path)
except:
    scd2_df = (incoming_df
                 .withColumn("__START_AT", col("update_timestamp"))
                 .withColumn("__END_AT", lit(None).cast("timestamp"))
                 .withColumn("__IS_CURRENT", lit(True)))
    scd2_df.write.format("delta").mode("overwrite").save(target_path)
    target = DeltaTable.forPath(spark, target_path)

# Step 3: Perform SCD2 Merge
(
    target.alias("t")
    .merge(
        incoming_df.alias("s"),
        "t.customer_id = s.customer_id AND t.__IS_CURRENT = true"
    )
    # Expire old record if data changed
    .whenMatchedUpdate(
        condition="t.name <> s.name OR t.city <> s.city",
        set={
            "__END_AT": "s.update_timestamp",
            "__IS_CURRENT": "false"
        }
    )
    # Insert new record if not matched
    .whenNotMatchedInsert(
        values={
            "customer_id": "s.customer_id",
            "name": "s.name",
            "city": "s.city",
            "update_timestamp": "s.update_timestamp",
            "__START_AT": "s.update_timestamp",
            "__END_AT": "NULL",
            "__IS_CURRENT": "true"
        }
    )
    .execute()
)


In [0]:
%sql
CREATE TABLE IF NOT EXISTS delta.`/Volumes/sivaadbuc/default/batch18test/customers_scd2` (
    customer_id INT,
    name STRING,
    city STRING,
    update_timestamp TIMESTAMP,
    __START_AT TIMESTAMP,
    __END_AT TIMESTAMP,
    __IS_CURRENT BOOLEAN
)
USING delta;


In [0]:
%sql
select *from delta.`/Volumes/sivaadbuc/default/batch18test/customers_scd2`

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW incoming_customers AS
SELECT
  _c0  AS customer_id,
  _c1 AS name,
  _c2 AS city,
  _c3 AS update_timestamp
FROM csv.`/Volumes/sivaadbuc/default/batch18test/customers_raw`;

In [0]:
%sql
MERGE INTO delta.`/Volumes/sivaadbuc/default/batch18test/customers_scd2` t
USING (
  SELECT 
    try_cast(customer_id AS BIGINT) AS customer_id,
    name,
    city,
    try_to_timestamp(update_timestamp, 'yyyy-MM-dd HH:mm:ss') AS update_timestamp
  FROM incoming_customers
) s
ON t.customer_id = s.customer_id AND t.__IS_CURRENT = true
WHEN MATCHED AND (t.name <> s.name OR t.city <> s.city) THEN
  UPDATE SET
    t.__END_AT = s.update_timestamp,
    t.__IS_CURRENT = false
WHEN NOT MATCHED THEN
  INSERT (
    customer_id, name, city, update_timestamp, __START_AT, __END_AT, __IS_CURRENT
  )
  VALUES (
    s.customer_id, s.name, s.city, s.update_timestamp, s.update_timestamp, NULL, true
  );